## Код для предсказания классов на test и создания файла submission.csv

In [5]:
from scipy.fft import fft
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

def generate_features(df): 
    '''
    Функция для сборки признаков из временных рядов
    Вход: Датафрейм с 2 временными рядами (values и dates)
    Выход: Датафрейм с извлеченными признаками (mean, std, min, max, median, value_range, fft_real_sum, fft_imag_sum, date_range)
    '''
    feature_list = []
    for _, row in df.iterrows():
        values = row['values']
        
        # Статистические признаки временного ряда
        features = {
            'id': row['id'],
            'mean': np.mean(values),
            'std': np.std(values),
            'min': np.min(values),
            'max': np.max(values),
            'median': np.median(values),
            'value_range': np.max(values) - np.min(values),  # Размах
            'fft_real_sum': np.sum(np.real(fft(values))),
            'fft_imag_sum': np.sum(np.imag(fft(values))),
        }
        
        # Признаки на основе даты
        dates = pd.to_datetime(row['dates'])  # Преобразование дат в формат datetime
        features.update({
            'date_range': (dates.max() - dates.min()).days  # Длительность ряда в днях
        })
        
        feature_list.append(features)
    
    return pd.DataFrame(feature_list)

In [6]:
import joblib
import numpy as np

# Загрузка модели и скейлера
model = joblib.load('trained_model.pkl')
scaler = joblib.load('scaler.pkl')

# Загрузка тестовой выборки и генерация признаков
test = pd.read_parquet('test.parquet')

# Генерация признаков и масштабирование
test_features = generate_features(test)
test_features = test_features.fillna(0)
test_features_scaled = scaler.transform(test_features.drop(columns=['id']))

# Предсказания
predictions = model.predict_proba(test_features_scaled)[:, 1]

# Создание файла submission.csv
submission = pd.DataFrame({'id': test_features['id'], 'score': predictions})
submission.to_csv('submission.csv', index=False)